In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Import libs

In [1]:
import pandas as pd
import json
import requests
import kagglehub
import re


# Import from kaggle

 https://www.kaggle.com/datasets/hugodarwood/epirecipes/data?

In [76]:
# Download latest version
path = kagglehub.dataset_download("hugodarwood/epirecipes")

print("Path to dataset files:", path)

100%|██████████| 11.3M/11.3M [00:01<00:00, 9.92MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/hugodarwood/epirecipes/versions/2


In [79]:
!cp /root/.cache/kagglehub/datasets/hugodarwood/epirecipes/versions/2/full_format_recipes.json .

In [109]:
filename = "full_format_recipes.json"
# filename = "as.json"
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

all_recipes = []
break_data_count = 0
data1 = pd.DataFrame(data)

recipe_data = {}
list_columns = ['directions', 'categories', 'ingredients']

for col in list_columns:
    if col in data1.columns:
        # Преобразуем списки в строки с разделителями
        data1[col] = data1[col].apply(lambda x: ' | '.join(x) if isinstance(x, list) else x)

# Конвертируем дату в datetime
data1 = data1.drop(columns=["date"], axis=1)
data1 = data1.rename(columns={'desc': 'description', "sodium":"salt"})

print(data1.shape)
display(data1.head())

(20130, 10)


,directions,fat,categories,calories,description,protein,rating,title,ingredients,salt
0,"1. Place the stock, lentils, celery, carrot, t...",7.0,Sandwich | Bean | Fruit | Tomato | turkey | Ve...,426.0,None,30.0,2.500,"Lentil, Apple, and Turkey Wrap",4 cups low-sodium vegetable or chicken stock |...,559.0
1,Combine first 9 ingredients in heavy medium sa...,23.0,Food Processor | Onion | Pork | Bake | Bastill...,403.0,This uses the same ingredients found in boudin...,18.0,4.375,Boudin Blanc Terrine with Red Onion Confit,"1 1/2 cups whipping cream | 2 medium onions, c...",1439.0
2,In a large heavy saucepan cook diced fennel an...,7.0,Soup/Stew | Dairy | Potato | Vegetable | Fenne...,165.0,None,6.0,3.750,Potato and Fennel Soup Hodge,"1 fennel bulb (sometimes called anise), stalks...",165.0
3,Heat oil in heavy large skillet over medium-hi...,NaN,Fish | Olive | Tomato | Sauté | Low Fat | Low ...,NaN,The Sicilian-style tomato sauce has tons of Me...,NaN,5.000,Mahi-Mahi in Tomato Olive Sauce,2 tablespoons extra-virgin olive oil | 1 cup c...,NaN
4,Preheat oven to 350°F. Lightly grease 8x8x2-in...,32.0,Cheese | Dairy | Pasta | Vegetable | Side | Ba...,547.0,None,20.0,3.125,Spinach Noodle Casserole,"1 12-ounce package frozen spinach soufflé, tha...",452.0


## analisys

In [111]:
data1[["calories","protein","salt","fat"]] = data1[["calories","protein","salt","fat"]].fillna(0)
data1 = data1[data1[["directions", 'categories', 'title', 'ingredients']].notna()]
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20130 entries, 0 to 20129
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   directions   0 non-null      object 
 1   fat          0 non-null      float64
 2   categories   0 non-null      object 
 3   calories     0 non-null      float64
 4   description  0 non-null      object 
 5   protein      0 non-null      float64
 6   rating       0 non-null      float64
 7   title        0 non-null      object 
 8   ingredients  0 non-null      object 
 9   salt         0 non-null      float64
dtypes: float64(5), object(5)
memory usage: 1.5+ MB


# Import from kaggle

https://www.kaggle.com/datasets/shuyangli94/food-com-recipes-and-user-interactions?select=RAW_interactions.csv

In [71]:
# Download latest version
path = kagglehub.dataset_download("shuyangli94/food-com-recipes-and-user-interactions")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/food-com-recipes-and-user-interactions


In [74]:
!cp /kaggle/input/food-com-recipes-and-user-interactions/RAW_recipes.csv .

In [75]:
data2 = pd.read_csv("RAW_recipes.csv")
data2.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
df['carbohydrate'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))
df['calories'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))
df['protein'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))
df['salt'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))
df['fat'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))


## analisys


In [99]:
# data3[["carbohydrate","calories","protein","salt","fat"]] = data3[["carbohydrate","calories","protein","salt","fat"]].fillna(0)
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 231637 entries, 0 to 231636
Data columns (total 12 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   name            231636 non-null  object
 1   id              231637 non-null  int64 
 2   minutes         231637 non-null  int64 
 3   contributor_id  231637 non-null  int64 
 4   submitted       231637 non-null  object
 5   tags            231637 non-null  object
 6   nutrition       231637 non-null  object
 7   n_steps         231637 non-null  int64 
 8   steps           231637 non-null  object
 9   description     226658 non-null  object
 10  ingredients     231637 non-null  object
 11  n_ingredients   231637 non-null  int64 
dtypes: int64(5), object(7)
memory usage: 21.2+ MB


# import from json

In [84]:
# URL для загрузки
url = "https://raw.githubusercontent.com/mneedham/bbcgoodfood/master/stream_all.json"
filename = "stream_all.json"  # Имя выходного файла

In [85]:
response = requests.get(url)
response.raise_for_status()  # Проверяем на ошибки HTTP

# Проверяем, что это действительно JSON
if 'application/json' not in response.headers.get('Content-Type', ''):
    print("Предупреждение: Сервер вернул не JSON-данные")

data = (response.text.replace('\"', "'")
        .replace('"{', "{")
        .replace('}"', "}")
        .replace("', ", '", ')
        .replace("': ", '": ')
        .replace(": '", ': "')
        .replace(": ['", ': ["')
        .replace(": {'", ': {"')
        .replace("'],", '"],')
        .replace("'}", '"}')
        .replace(", '", ', "')
        .replace("'page", '"page')
        .split("\n")
)
# Сохраняем в файл
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=4)

print(f"Данные успешно сохранены в файл {filename}")

Предупреждение: Сервер вернул не JSON-данные
Данные успешно сохранены в файл stream_all.json


In [90]:
filename = "stream_all.json"
with open(filename, 'r', encoding='utf-8') as f:
    data = json.load(f)

all_recipes = []
break_data_count = 0
for item in data:
    try:
        item = json.loads(item)
    except:
        break_data_count+=1
        continue
    recipe_data = {}

    # Извлекаем данные из article
    article = item['page']['article']
    recipe_data.update({
        'author': article['author'],
        'description': article['description'],
    })

    # Извлекаем данные из recipe
    recipe = item['page']['recipe']
    recipe_data.update({
        'cooking_time': recipe['cooking_time']/60,
        'prep_time': recipe['prep_time']/60,
        'categories': ' | '.join(recipe['keywords']+recipe['collections']+
         recipe['diet_types']+[recipe['cusine'],recipe['skill_level']]),
        'ingredients': ' | '.join(recipe['ingredients']),
        'cuisine': recipe['cusine'],
        'skill_level': recipe['skill_level'],
        'rating': float(recipe['ratings'])/20,
    })
    for elem in recipe['nutrition_info']:
        elem = elem.split(" ")
        if elem[0] in ['Carbohydrate','Protein','Salt','Fat']:
            name, num = elem[0].lower(), elem[1][:-1]
        elif elem[-1]=="calories":
            name, num = "calories", elem[1]
        else:
            continue
        recipe_data.update({name: float(num)})

    # Добавляем оставшиеся поля
    recipe_data['title'] = item['page']['title']

    all_recipes.append(recipe_data)

# Создаём DataFrame
data3 = pd.DataFrame(all_recipes)
print(f"Skip {break_data_count} reciept because break data")

Skip 31 reciept because break data


In [91]:
print(data3.shape)
display(data3.head())
data3.to_csv("./stream_all.csv", sep="\t")

(11606, 15)


,author,description,cooking_time,prep_time,categories,ingredients,cuisine,skill_level,rating,carbohydrate,calories,protein,salt,fat,title
0,Tony Tobin,Keep some couscous in your cupboard as a stand...,0.0,15.0,Pesto | Tomato | Mozzarella | Couscous | Tobin...,couscous | pesto | vegetable stock | mozzarell...,Italian,Easy,4.75,32.0,363.0,13.0,1.02,22.0,Tomato &amp; mozzarella couscous salad
1,Jane Hornby,One of the simplest ways to showcase summer fr...,30.0,15.0,Vit C | Vitamin c | 1 of 5 a day | Baked fruit...,white wine | clear honey | nectarine | amarett...,British,Easy,4.80,35.0,253.0,5.0,0.10,7.0,Baked nectarines &amp; raspberries with almond...
2,Sarah Cook,For the ultimate indulgence look no further th...,55.0,60.0,Chocolate | Indulgent | Four-layered | Caramel...,butter | golden caster sugar | self-raising fl...,British,More effort,4.45,99.0,1036.0,15.0,1.25,67.0,Chocolate &amp; caramel ombre cake
3,Good Food,This is a really easy meal to cook for friends...,0.0,0.0,Chicken breast | Chicken breasts | Chicken | B...,chicken breast | goat's cheese | thyme | tomat...,British,Easy,4.45,5.0,330.0,40.0,1.24,16.0,"Chicken, goat&#039;s cheese &amp; cherry tomat..."
4,Jennifer Joyce,"A spicy, smoky&nbsp;chicken filling in soft to...",10.0,10.0,chicken tacos | low calorie dinner | easy taco...,skinless boneless chicken thigh | vegetable oi...,Mexican,Easy,4.80,37.0,392.0,30.0,0.60,13.0,Chipotle chicken tacos with pineapple salsa


## analisys

In [98]:
data3[["carbohydrate","calories","protein","salt","fat"]] = data3[["carbohydrate","calories","protein","salt","fat"]].fillna(0)
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11606 entries, 0 to 11605
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   author        11606 non-null  object 
 1   description   11606 non-null  object 
 2   cooking_time  11606 non-null  float64
 3   prep_time     11606 non-null  float64
 4   categories    11606 non-null  object 
 5   ingredients   11606 non-null  object 
 6   cuisine       11606 non-null  object 
 7   skill_level   11606 non-null  object 
 8   rating        11606 non-null  float64
 9   carbohydrate  11606 non-null  float64
 10  calories      11606 non-null  float64
 11  protein       11606 non-null  float64
 12  salt          11606 non-null  float64
 13  fat           11606 non-null  float64
 14  title         11606 non-null  object 
dtypes: float64(8), object(7)
memory usage: 1.3+ MB


(11606, 17)

# Concate datas

### Add new column

In [ ]:
meat = ['meat', 'gammon', 'mutton', 'beef', 'pork', 'deer', 'camel', 'veal', 'lamb', 'goat', 'rabbit', 'turkey', 'steak', 'goose', 'duck', 'venison', 'elk', 'moose', 'loin', 'bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', 'chicken', 'crackling']
milk = ['kaymak', 'milk', 'cream', 'butter', 'yogurt', 'ymer', 'viili', 'kefir', 'kumis', 'amasi', 'mursik', 'quark', 'acidophiline', 'matzoon', 'skyr', 'yogurt', 'qatyq', 'ryazhenka', 'varenets', 'ayran', 'doogh', 'lassi', 'leben', 'cheese', 'custard', 'casein', 'dairy products', 'milk products']
alcohol = ["'wine', 'beer', liqueur", "champagne", "ale", "gin", "cider", "brandy", "whisky", "stout", "lager", "rum", "vodka", "tequila", "absinthe", "alcohol", "ethyl", 'pastis', 'sherry', "moonshine"]
pig = ['bacon', 'ham', 'baked ham', 'frankfurter', 'sausage', "pork", "pig", 'gammon', 'crackling']
citrus = ['orange', 'grapefruit', 'lemon', 'tangerine', 'lime', 'kumquat', 'pomelo', 'yuzu', 'ugli', 'sudachi', 'pummelo', 'tangelo', 'mandarin']
fruts = ['apple', 'strawberries', 'apricot','acai','ackee','almond','amla','avocado','banana','blackberry','blackcurrant','blueberry','boysenberry','breadfruit','cactus','calabash','cantaloupe','carambola ','starfruit','carob','cashew','casimiroa','ceriman','chayote','cherimoya','cherry','chestnut','chinotto','citron','clementine','coconut','cocona','cranberry','custard','damson','date','dragonfruit','durian','elderberry','feijoa','fig','gooseberry','grape','grapefruit','guanabana (soursop)','guava','honeyberry','horned melon','huckleberry','jabuticaba','jackfruit','jambolan','jujube','juniper berry','karonda','key lime','kiwi','kumquat','langsat','lemon','lime','longan','loquat','lychee','mango','mangosteen','marionberry','medlar','melon','mulberry','nectarine','noni','orange','oroblanco','osage orange','papaya','passionfruit','pawpaw','peach','pear','persimmon','pineapple','pineberry','plum','pomegranate','pomelo','quince','raisin','rambutan','raspberry','redcurrant','rhubarb','rose apple','rowanberry','salak','saskatoon berry','sea buckthorn','sharon fruit','sloe','sour cherry','star apple','strawberry','sugar apple','surinam cherry','tamarillo','tangerine','tangelo','tangerillo','ugli fruit','vanilla bean','watermelon']
fish = ['salmon', 'trout', 'tuna', 'catfish', 'pike', 'sturgeon', 'starlet', 'dorado', 'sea bass', 'eel', 'herring', 'mackerel', 'cod', 'haddock', 'hake', 'plaice', 'flounder', 'sardine', 'perch', 'grouper', 'pike-perch', 'carp', 'bream', 'burbot', 'navaga', 'mullet', 'anchovy']
animal_root = fish + meat + milk + ["egg"]
nuts = ['coconut', "soy", 'chestnut', 'walnut', 'peanut', 'cola', 'almond', 'pistachio', 'hazelnut', 'pecan', 'brazil nut', 'pine nut', 'nutmeg', 'filbert', 'sesame', 'sunflower seeds', 'pumpkin seeds', 'water chestnut', 'macadamia nut', 'cashew', 'pignoli']

contra_dict = {
    "Halal": pig+alcohol,
    "Kashrut": [meat, milk, pig+alcohol],
    "Christian": meat+milk+alcohol+fish+["egg, bread"],
    "Vegan": animal_root,
    "Vegeterian": meat+fish,
    "Hindu": pig+["beef", "veal"],
    "Buckwheat": ["buckwheat", "wheat"],
    "Balsam of Peru": ['coffee','flavored tea','wine','beer','gin','liqueurs','apéritifs','cola','juice','citrus','marmalade','tomatoes','sauces','ketchup','chocolate','vanilla','pudding','ice cream','gum','candy'],
    "Celery": ['parsley', 'culantro', 'dill', 'coriander', 'fennel', 'cumin', 'caraway', 'celery'],
    "Egg": ["egg"],
    "Fish": fish,
    "Fruit": fruts+['tomato'],
    "citrus": citrus,
    "Garlic": ['garlic'],
    "Oats": ['oats'],
    "Corn": ["corn"],
    "Mustard": ['mustard'],
    "Milk": ['milk', "soy"],
    "Nut": nuts,
    "Poultry Meat": ['chicken', 'turkey', 'squab'],
    "Red Meat": ['beef', 'venison', 'lamb', 'pork'],
    "rice": ['rice'],
    "seafood": ['shellfish', 'mussels', 'caviar', 'shrimps', 'prawns', 'oysters', 'squids', 'crabs', 'crab', 'lobsters', 'scallops', 'octopus', 'crayfish', 'cuttlefish'],
    "Sulfites": ['zante currants', 'mulberry', 'physalis', 'aronia', 'chokeberries', 'sea-buckthorn', 'kumquats', 'white raisins', 'goji', 'cherries', 'cranberries', 'sour cherries', 'barberries', "apricots", "fig", "peache", "pear", "prune", "raisin",],
}

In [ ]:
cols = ["categories",	"ingredients", "description"]
df['combined'] = df[cols].apply(lambda row: ' _ '.join(row.values.astype(str)), axis=1)

def find_matches(text, category_dict):
    if pd.isna(text):  # Проверка на NaN
        return []
    found_categories = set()
    text_lower = str(text).lower()  # Преобразуем в строку и в нижний регистр
    for category, keywords in category_dict.items():
        if category=="Kashrut":
            kashrut = True
            for keyword in keywords[-1]:
                if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                    found_categories.add(category)
                    break
            for keyword in keywords[0]:
                if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                    kashrut=False
                    break
            if not kashrut:
                for keyword in keywords[1]:
                    if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                        found_categories.add(category)
                        break
        else:
            for keyword in keywords:
                if re.search(r'\b' + re.escape(keyword) + r'\b', text_lower):
                    found_categories.add(category)
                    break  # Прерываем после первого совпадения в категории

    return "| ".join(list(found_categories))


df['contra'] = df['combined'].apply(lambda x: find_matches(x, contra_dict))

In [ ]:
df.shape
